# Assignment A1b - Functions and Computation
---
### Ryan Lin


## Imports

In [54]:
#imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


## 1a sinewave

In [55]:
#import sinewave
from sinewave import sinewave

In [56]:
sinewave(0.0, f=5, d=0.05)


np.float64(-0.04997916927067833)

## 1b gabor


spatial signal

In [57]:
def gabor(t, f=1.0, sigma=1.0, d=0.0, a=1.0):
    gaussian_envolope = np.exp(-t**2 / (2 * sigma**2))
    sinusoid = np.cos(2 * np.pi * f * t + d)
    return a * gaussian_envolope * sinusoid



In [58]:
def gabore(t, f=1.0, sigma=1.0, a=1.0):
    return gabor(t, f, sigma, 0.0, a) # delay is 0 for cosine

def gaboro(t, f=1.0, sigma=1.0, a=1.0):
    return gabor(t, f, sigma, np.pi/2, a) # delay is pi/2 for sine

In [59]:
def gabor_norm(f=1.0, sigma=1.0, fs=1000):
    t = np.linspace(-3 * sigma, 3 * sigma, int(6 * sigma * fs))
    g = gabor(t, f, sigma, 0.0, 1.0)
    return 1 / np.sqrt(np.sum(g**2))

def gabore_norm(f=1.0, sigma=1.0, fs=1000):
    return gabor_norm(f, sigma, fs)

def gaboro_norm(f=1.0, sigma=1.0, fs=1000):
    t = np.linspace(-3 * sigma, 3 * sigma, int(6 * sigma * fs))
    g = gabor(t, f, sigma, np.pi/2, 1.0)
    return 1 / np.sqrt(np.sum(g**2))

## 1c gammatone

cochlear signal

In [60]:
def erb(f):
    return 24.7*((4.37 * f)/1000) + 1

def bandwidth(f):
    return 1.019 * erb(f)

def gammatone(t, f=1.0, n=4, d=0.0, a=1.0):
    t = np.maximum(t,0) # t >= 0
    b = bandwidth(f)
    return a * (t**(n-1)) * np.exp(-2 * np.pi * b * t) * sinewave(t, f, np.pi/2) #cosine is 2 pi

def gammatone_norm(f=1.0, n=4, fs=1000):
    t = np.linspace(0, 0.1, int(0.1 * fs))  # 0 to 0.1 seconds
    g = gammatone(t, f, n, 0.0, 1.0)
    return 1 / np.sqrt(np.sum(g**2))


## 2a localmaxima

In [61]:
def localmaxima(signal):
    """
    Find the indices of local maxima in a 1D signal.

    Parameters:
    signal : array-like
        The input 1D array.

    Returns:
    list
        A list of indices where local maxima occur.
    """
    # Convert input to NumPy array for easy slicing
    signal = np.asarray(signal)

    # Initialize an empty list to store indices of local maxima
    maxima_indices = []

    # Iterate through the array, ignoring the first and last elements (edges)
    for i in range(1, len(signal) - 1):
        # Check if the current element is a local maximum
        if signal[i - 1] < signal[i] > signal[i + 1]:
            maxima_indices.append(i)

    return maxima_indices


## 2b crossings

In [62]:
def threshold_crossings(signal, threshold, direction="both"):
    """
    Find the indices of threshold crossings in a 1D signal.

    Parameters:
    signal : array-like
        The input 1D array.
    threshold : float
        The threshold value to check crossings against.
    direction : str, optional
        The direction of the crossings to detect:
        - "negpos": Crossings from below the threshold to equal to or greater than the threshold.
        - "posneg": Crossings from equal to or greater than the threshold to below the threshold.
        - "both": Crossings in either direction.
        Default is "both".

    Returns:
    numpy.ndarray
        An array of indices where threshold crossings occur.
    """
    # Convert signal to a NumPy array
    signal = np.asarray(signal)
    
    # Initialize crossings
    crossings = []

    # Compute the conditions for each direction
    if direction == "negpos":
        # Crossing from below to above or equal
        crossings = np.where((signal[:-1] < threshold) & (signal[1:] >= threshold))[0] + 1
    elif direction == "posneg":
        # Crossing from above or equal to below
        crossings = np.where((signal[:-1] >= threshold) & (signal[1:] < threshold))[0] + 1
    elif direction == "both":
        # Crossing in either direction
        negpos_crossings = np.where((signal[:-1] < threshold) & (signal[1:] >= threshold))[0] + 1
        posneg_crossings = np.where((signal[:-1] >= threshold) & (signal[1:] < threshold))[0] + 1
        crossings = np.sort(np.concatenate((negpos_crossings, posneg_crossings)))
    else:
        raise ValueError("Invalid direction. Must be 'negpos', 'posneg', or 'both'.")

    return crossings


## 2c envelope

In [63]:
def envelope(y, nblocks=None):
    """
    Compute the lower and upper bounds of a waveform in `nblocks` blocks.

    Parameters:
    y : array-like
        The input waveform or vector.
    nblocks : int, optional
        Number of blocks to divide `y` into. Default is 1/10th the length of `y`.

    Returns:
    tuple:
        - ylower : numpy.ndarray
            Array of lower bounds for each block.
        - yupper : numpy.ndarray
            Array of upper bounds for each block.
        - blockindices : numpy.ndarray
            Array of starting indices for each block.
    """
    # Convert input to a NumPy array
    y = np.asarray(y)
    
    # Set default for `nblocks` if not provided
    if nblocks is None:
        nblocks = max(1, len(y) // 10)  # Default to 1/10th the length of `y`

    # Compute the size of each block
    block_size = len(y) // nblocks

    # Compute the starting indices for each block
    blockindices = np.arange(0, len(y), block_size)

    # Initialize lower and upper bounds arrays
    ylower = []
    yupper = []

    # Compute lower and upper bounds for each block
    for i in range(len(blockindices)):
        start = blockindices[i]
        end = blockindices[i + 1] if i + 1 < len(blockindices) else len(y)  # Handle last block
        block = y[start:end]
        ylower.append(np.min(block))  # Lower bound
        yupper.append(np.max(block))  # Upper bound

    return np.array(ylower), np.array(yupper), blockindices
